In [1]:
import time
import pandas as pd
from multiprocessing import Pool
import math
import os
import shutil
import matplotlib.pyplot as plt
from scipy.io import readsav
import glob
import statistics
from aetherpy.io import read_routines
from math import cos, radians, sin, sqrt
from scipy import spatial, signal
from glob import glob
from datetime import datetime
from datetime import timedelta
from struct import unpack
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from pylab import cm
import fnmatch

from aetherpy.io import read_routines as rr

import sys

from scipy.interpolate import LinearNDInterpolator, interp1d

from utility_programs.read_routines import SAMI
from utility_programs.filters import make_fits

import xarray as xr
import importlib
import gc

import cartopy.crs as ccrs

import xesmf as xe

In [2]:
sami_data_path = "/petastore/phil/GITM/cheyenne_runs/FullAmp/sami-gitm-coupled/"
sami_og_vars = {
    'deneu.dat': 'edens', }

geo_grid_files = {
    'glat': 'glatu.dat', 'glon': 'glonu.dat', 'alt': 'zaltu.dat',
    'mlat': 'blatu.dat', 'mlon': 'blonu.dat', 'malt': 'baltu.dat'
}

In [3]:
nz, nf, nlt, nt = SAMI.get_grid_elems_from_parammod(sami_data_path)
times = SAMI.make_times(nt, sami_data_path, datetime(2011, 5, 20))

In [6]:
grid = {}

for f in geo_grid_files:
    file = open(os.path.join(sami_data_path, geo_grid_files[f]), 'rb')
    raw = np.fromfile(file, dtype='float32')[1:-1].copy()
    file.close()

    grid[f] = raw.reshape(nlt, nf, nz).copy()

In [7]:
ds = xr.Dataset(
    coords=dict(
        time=(('time'), times),
        mlat=(('nlt', 'nf', 'nz'), grid['mlat']),
        mlon=(('nlt', 'nf', 'nz'), grid['mlon'].round(2)),
        malt=(('nlt', 'nf', 'nz'), grid['malt']),
        glat=(('nlt', 'nf', 'nz'), grid['glat']),
        glon=(('nlt', 'nf', 'nz'), grid['glon']),
        alt=(('nlt', 'nf', 'nz'), grid['alt'],)
    )
)

In [12]:
dimnames = ('time', 'nlt', 'nf', 'nz')

for fname in sami_og_vars.keys():
    curr_arr = np.zeros((len(times), nlt, nf, nz))
    with open(os.path.join(sami_data_path, fname), 'rb') as f:
        t_ins = 0
        for t in range(len(times)):
            raw = np.fromfile(f, dtype='float32',
                              count=(nz*nf*nlt)+2)[1:-1]
            curr_arr[t_ins] = raw.reshape(nlt, nf, nz).copy()
            t_ins += 1

        ds[sami_og_vars[fname]] = (dimnames, curr_arr)

In [13]:
# ds = ds.isel(time=34)

In [14]:
ds

<xarray.Dataset>
Dimensions:  (time: 624, nlt: 80, nf: 72, nz: 256)
Coordinates:
  * time     (time) datetime64[ns] 2011-05-20 ... 2011-05-22T03:55:00
    mlat     (nlt, nf, nz) float32 -1.99 -1.988 -1.986 ... 87.68 87.68 87.68
    mlon     (nlt, nf, nz) float32 2.25 2.25 2.25 2.25 ... 357.8 357.8 357.8
    malt     (nlt, nf, nz) float32 6.451e+03 6.451e+03 ... 6.458e+03 6.456e+03
    glat     (nlt, nf, nz) float32 -13.74 -13.74 -13.74 ... 80.04 80.04 80.04
    glon     (nlt, nf, nz) float32 290.3 290.3 290.3 290.3 ... 277.6 277.6 277.6
    alt      (nlt, nf, nz) float32 80.02 80.03 80.05 80.06 ... 89.94 87.09 84.4
Dimensions without coordinates: nlt, nf, nz
Data variables:
    edens    (time, nlt, nf, nz) float64 1.03e+03 1.03e+03 ... 29.3 29.3

In [18]:
ds.plot.scatter(x='glon', y='alt', col=ds.edens.where(ds.alt.isin([630, 650])))

TypeError: unhashable type: 'Dataset'

In [29]:
ds.isel(time=450)

<xarray.Dataset>
Dimensions:  (nlt: 80, nf: 72, nz: 256)
Coordinates:
    time     datetime64[ns] 2011-05-21T13:30:00
    mlat     (nlt, nf, nz) float32 -1.99 -1.988 -1.986 ... 87.68 87.68 87.68
    mlon     (nlt, nf, nz) float32 2.25 2.25 2.25 2.25 ... 357.8 357.8 357.8
    malt     (nlt, nf, nz) float32 6.451e+03 6.451e+03 ... 6.458e+03 6.456e+03
    glat     (nlt, nf, nz) float32 -13.74 -13.74 -13.74 ... 80.04 80.04 80.04
    glon     (nlt, nf, nz) float32 290.3 290.3 290.3 290.3 ... 277.6 277.6 277.6
    alt      (nlt, nf, nz) float32 80.02 80.03 80.05 80.06 ... 89.94 87.09 84.4
Dimensions without coordinates: nlt, nf, nz
Data variables:
    edens    (nlt, nf, nz) float64 48.32 48.32 45.17 44.27 ... 124.9 30.47 30.47

In [28]:
ds.alt.max()

<xarray.DataArray 'alt' ()>
array(5624980., dtype=float32)

In [ ]:
np.unique(ds.mlon)

array([  2.25,   6.75,  11.25,  15.75,  20.25,  24.75,  29.25,  33.75,
        38.25,  42.75,  47.25,  51.75,  56.25,  60.75,  65.25,  69.75,
        74.25,  78.75,  83.25,  87.75,  92.25,  96.75, 101.25, 105.75,
       110.25, 114.75, 119.25, 123.75, 128.25, 132.75, 137.25, 141.75,
       146.25, 150.75, 155.25, 159.75, 164.25, 168.75, 173.25, 177.75,
       182.25, 186.75, 191.25, 195.75, 200.25, 204.75, 209.25, 213.75,
       218.25, 222.75, 227.25, 231.75, 236.25, 240.75, 245.25, 249.75,
       254.25, 258.75, 263.25, 267.75, 272.25, 276.75, 281.25, 285.75,
       290.25, 294.75, 299.25, 303.75, 308.25, 312.75, 317.25, 321.75,
       326.25, 330.75, 335.25, 339.75, 344.25, 348.75, 353.25, 357.75],
      dtype=float32)

In [ ]:
%time mask1 = np.where((ds.mlon==2.25))
%time mask2 = np.where( (ds.alt > 600))
%time mask3 = np.where( (ds.alt < 630))

CPU times: user 4.89 ms, sys: 2.84 ms, total: 7.73 ms
Wall time: 6.15 ms
CPU times: user 4.27 ms, sys: 0 ns, total: 4.27 ms
Wall time: 4.04 ms
CPU times: user 4.81 ms, sys: 3.73 ms, total: 8.54 ms
Wall time: 8.56 ms


In [ ]:
mask1

(array([    0,     1,     2, ..., 18429, 18430, 18431]),)

In [ ]:
%time mask1 = (ds.mlon==2.25)
%time mask2 = (ds.alt > 600)
%time mask3 = (ds.alt < 630)

CPU times: user 4.28 ms, sys: 311 µs, total: 4.59 ms
Wall time: 2.69 ms
CPU times: user 2.56 ms, sys: 2.46 ms, total: 5.02 ms
Wall time: 4.05 ms
CPU times: user 2.03 ms, sys: 0 ns, total: 2.03 ms
Wall time: 2.04 ms


In [ ]:
mask1

<xarray.DataArray 'mlon' (mlon: 1474560)>
array([ True,  True,  True, ..., False, False, False])
Coordinates:
    time     datetime64[ns] 2011-05-20T02:50:00
  * mlon     (mlon) float32 2.25 2.25 2.25 2.25 2.25 ... 357.8 357.8 357.8 357.8

In [ ]:
ma = ds.alt.isin([600,  630]) & ds.mlon.isin([2.25])

In [ ]:
ma

<xarray.DataArray 'alt' (alt: 1474560)>
array([False, False, False, ..., False, False, False])
Coordinates:
    time     datetime64[ns] 2011-05-20T02:50:00
  * alt      (alt) float32 80.02 80.03 80.05 80.06 ... 92.96 89.94 87.09 84.4